In [2]:
print("OK!")

OK!


In [4]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

/home/kirthik-venkatram/anaconda3/envs/mchatbot/lib/python3.8/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [63]:
PINECONE_API_KEY = "5faa92b3-bf34-4bf9-b9d3-6de6ba145507"
PINECONE_API_ENV = "aws-starter"

In [7]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [9]:
extracted_data = load_pdf("data/")

In [10]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [11]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [12]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [13]:
embeddings = download_hugging_face_embeddings()

/home/kirthik-venkatram/anaconda3/envs/mchatbot/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)
/home/kirthik-venkatram/anaconda3/envs/mchatbot/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [15]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [68]:

from pinecone import Pinecone

pc = Pinecone(api_key="5faa92b3-bf34-4bf9-b9d3-6de6ba145507")
index = pc.Index("medical-chatbot")

def create_embeddings_and_upsert(text_chunks, embeddings, index):
  """Creates embeddings for text chunks and upserts them to Pinecone with error handling.

  Args:
      text_chunks: A list of text chunks.
      embeddings: An embedding model.
      index: A Pinecone index.
  """

  embeddings_list = []
  for i, chunk in enumerate(text_chunks):
      try:
          embedding = embeddings.embed(chunk.page_content)
          embeddings_list.append((str(i), embedding))
      except Exception as e:
          print(f"Error embedding chunk {i}: {e}")
          continue  # Skip problematic chunks

  if embeddings_list:
      try:
          index.upsert(vectors=embeddings_list)
      except PineconeApiException as e:
          print(f"Error upserting embeddings: {e}")
          # Handle upsert failure (e.g., retry or log the error)
